In [1]:
import utils
import os

In [2]:
HP_PATH = 'data/hp.obo'

LABITEMS_PATH = 'data/D_LABITEMS.csv'
ANNOTATION_PATH = 'data/loinc2hpo-annotations.tsv'
LABEVENTS_PATH = 'data/LABEVENTS.csv'

LABEVENTS_HPO_PATH = 'data/OUT_LABEVENTS_HPO.csv'

ICD_TO_HPO_PATH = 'data/semiautomatic_ICD-pheno.tsv'

In [6]:
def download_if_not_already(path: str, url: str, extract: bool=False):
    if not os.path.isfile(path):
        print(f'downloading {path}')
        import requests  # for downloading the hpo
        r:requests.Response = requests.get(url)
        b:bytes  = r.content
        if extract:
            print(f'extracting {path}')
            import gzip
            b = gzip.decompress(b)
        with open(path, 'wb') as f:
            f.write(b)


In [7]:
download_if_not_already(HP_PATH, 'http://purl.obolibrary.org/obo/hp.obo')
download_if_not_already(ANNOTATION_PATH, 'https://raw.githubusercontent.com/TheJacksonLaboratory/loinc2hpoAnnotation/master/loinc2hpo-annotations.tsv')
download_if_not_already(LABITEMS_PATH, 'https://physionet.org/files/mimiciii-demo/1.4/D_LABITEMS.csv')
download_if_not_already(LABEVENTS_PATH, 'https://physionet.org/files/mimiciii-demo/1.4/LABEVENTS.csv')
download_if_not_already(ICD_TO_HPO_PATH , 'https://zenodo.org/record/4726714/files/semiautomatic_ICD-pheno.txt.gz?download=1', extract = True)


downloading data/semiautomatic_ICD-pheno.tsv
extracting data/semiautomatic_ICD-pheno.tsv


In [ ]:
if not os.path.isfile(LABEVENTS_HPO_PATH):
    utils.add_hpo_information(
        LABITEMS_PATH, LABEVENTS_PATH,  ANNOTATION_PATH, LABEVENTS_HPO_PATH)


In [ ]:
hpo = utils.read_hpo_from_obo(HP_PATH)


In [ ]:
g = utils.make_graph_to_depth(hpo.entries_by_id['HP:0000001'], 1)
print(hpo.root.other_tags)
g